In [1]:
from lxml import etree
from display_xml import XML

from io import StringIO
from collections import Counter

# for rendering xml docs as nice collapsible trees
from xml_tree import render_tree

def fast_iter(context):
    """
    http://lxml.de/parsing.html#modifying-the-tree
    Based on Liza Daly's fast_iter
    http://www.ibm.com/developerworks/xml/library/x-hiperfparse/
    See also http://effbot.org/zone/element-iterparse.htm
    """
    for event, elem in context:
        yield event, elem
        # It's safe to call clear() here because no descendants will be
        # accessed
        elem.clear()
        # Also eliminate now-empty references from the root node to elem
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
    del context

# Parsing Clinvar Full Release

In [2]:
record_types = Counter()
genes = Counter()

# first, load the latest clinvar from a static file
with open("../data/clinvar/ClinVarFullRelease_2019-06.xml", "rb") as fp:
    ctx = etree.iterparse(fp, events=("end",), tag="ClinVarSet")

    for idx, (action, elem) in enumerate(fast_iter(ctx)):
        if action == 'end':
            title = elem.find('Title').text
            inferred_gene = title.split(':')[0]
            
            try:
                record_status = elem.find('ReferenceClinVarAssertion').find('RecordStatus').text
                record_types[record_status] += 1
            except AttributeError:
                print("Couldn't find required element, continuing...")
                raise
            
            try:
                gene_symbol = elem.find('ReferenceClinVarAssertion').find('MeasureSet').find('Measure').find('MeasureRelationship').find('Symbol').find('ElementValue').text
                genes[gene_symbol] += 1
            except AttributeError:
                print("Couldn't find required element for gene %s, continuing..." % inferred_gene)
                print(etree.tostring(elem))
                raise
                
        if idx > 10000:
            break

    print(record_types)
    print(genes)

Couldn't find required element for gene NM_000339.2(SLC12A3), continuing...
b'<ClinVarSet xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ID="25328796">\n  <RecordStatus>current</RecordStatus>\n  <Title>NM_000339.2(SLC12A3):c.[1849C&gt;T];[1919A&gt;G] AND Familial hypokalemia-hypomagnesemia</Title>\n  <ReferenceClinVarAssertion DateCreated="2016-10-12" DateLastUpdated="2018-01-06" ID="619044">\n    <ClinVarAccession Acc="RCV000256196" Version="1" Type="RCV" DateUpdated="2018-01-06"/>\n    <RecordStatus>current</RecordStatus>\n    <ClinicalSignificance DateLastEvaluated="2016-09-06">\n      <ReviewStatus>criteria provided, single submitter</ReviewStatus>\n      <Description>Pathogenic</Description>\n    </ClinicalSignificance>\n    <Assertion Type="variation to disease"/>\n    <AttributeSet>\n      <Attribute Type="ModeOfInheritance" integerValue="263">Autosomal recessive inheritance</Attribute>\n    </AttributeSet>\n    <ObservedIn>\n      <Sample>\n        <Origin>unknown</Origi

AttributeError: 'NoneType' object has no attribute 'find'

## More light parsing of ClinVar: full

In [ ]:
gene_set = set(['BRAF'])

# first, load the latest clinvar from a static file
with open("../data/clinvar/ClinVarFullRelease_2019-06.xml", "rb") as fp:
    ctx = etree.iterparse(fp, events=("end",), tag="ClinVarSet")

    for idx, (action, elem) in enumerate(fast_iter(ctx)):
        if action == 'end':
#             if gene_set:
#                 these_genes = set(elem.xpath("InterpretedRecord/*/GeneList/Gene/@Symbol"))
#                 if gene_set.isdisjoint(these_genes):
#                     continue
                    
            # if elem.attrib['VariationID'] == '13961':
            q = elem
            break

            # title = elem.find('Title').text
            # inferred_gene = title.split(':')[0]


In [ ]:
render_tree(q)

# Parsing ClinVar: Variation

In [16]:
%%bash
ls ../data/clinvar

ClinVarFullRelease_2019-06.xml
ClinVarVariationRelease_2019-09.xml
clinvar_13961.xml
clinvar_16609.xml
parse_variations.py
tsv
variant_summary.txt
variation_archive.xsd


In [13]:
# BRAF V600E is 13961
# EGFR L858R is 16609

VARIANT_ID='16609'
USE_CACHED_FILE=True
VARIATION_FILE = "../data/clinvar/clinvar_%s.xml" % VARIANT_ID if USE_CACHED_FILE else "../data/clinvar/ClinVarVariationRelease_2019-09.xml"

# first, load the latest clinvar from a static file
with open(VARIATION_FILE, "rb") as fp:
    ctx = etree.iterparse(fp, events=("end",), tag="VariationArchive")

    for idx, (action, elem) in enumerate(fast_iter(ctx)):
        if action == 'end':                    
            if elem.attrib['VariationID'] == str(VARIANT_ID):
                q = elem
                break
                
# cache the found variant so we don't have to linear-search for it again
with open("../data/clinvar/clinvar_%s.xml" % VARIANT_ID, "wb") as fp:
    fp.write(etree.tostring(q))

In [14]:
render_tree(q)

In [39]:
q.xpath('//HGVSlist/HGVS/ProteinExpression/Expression/text()')

['NP_001333826.1:p.Leu813Arg',
 'NP_001333827.1:p.Leu858Arg',
 'NP_001333828.1:p.Leu813Arg',
 'NP_001333829.1:p.Leu805Arg',
 'NP_001333870.1:p.Leu591Arg',
 'NP_005219.2:p.Leu858Arg',
 'LRG_304p1:p.Leu858Arg',
 'P00533:p.Leu858Arg']

In [40]:
q.xpath('//ProteinChange/text()')

['L858R', 'L591R', 'L805R', 'L813R']

In [5]:
len(q.xpath('//RCVList/RCVAccession'))

23

In [6]:
len(q.xpath('//ClinicalAssertionList/ClinicalAssertion'))

26

In [7]:
sorted([
    x.xpath(".//InterpretedCondition/@*") for x in q.xpath('//RCVList/RCVAccession')
])

[[],
 ['MedGen', 'C0006118'],
 ['MedGen', 'C0007131'],
 ['MedGen', 'C0009375'],
 ['MedGen', 'C0009404'],
 ['MedGen', 'C0017636'],
 ['MedGen', 'C0025202'],
 ['MedGen', 'C0026764'],
 ['MedGen', 'C0027651'],
 ['MedGen', 'C0151779'],
 ['MedGen', 'C0152013'],
 ['MedGen', 'C0238198'],
 ['MedGen', 'C0238463'],
 ['MedGen', 'C0677865'],
 ['MedGen', 'C0684249'],
 ['MedGen', 'C0699790'],
 ['MedGen', 'C1168401'],
 ['MedGen', 'C1266158'],
 ['MedGen', 'C1275081'],
 ['MedGen', 'C1336078'],
 ['MedGen', 'C2674727'],
 ['MedGen', 'CN236629'],
 ['MedGen', 'CN517202']]

In [8]:
sorted([
    x.xpath(".//Trait/XRef/@*")
    for x in q.xpath('//ClinicalAssertionList/ClinicalAssertion')
])

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 ['MeSH', 'C535575'],
 ['MeSH', 'C536915'],
 ['MeSH', 'C538231'],
 ['MeSH', 'C538614'],
 ['MeSH', 'C562393'],
 ['MeSH', 'D001932'],
 ['MeSH', 'D002289'],
 ['MeSH', 'D003110'],
 ['MeSH', 'D005909'],
 ['MeSH', 'D008545'],
 ['MeSH', 'D009101'],
 ['MeSH', 'D009369'],
 ['MeSH', 'D010051'],
 ['MeSH', 'D015179'],
 ['MeSH', 'D046152'],
 ['MedGen', 'C0007131', 'CUI'],
 ['MedGen', 'C0677865', 'CUI'],
 ['MedGen', 'C1275081', 'CUI'],
 ['OMIM', '211980', 'MIM']]

### Aside: examining specific variants

In [9]:
def get_variant(variation_id):
    VARIATION_FILE = "../data/clinvar/ClinVarVariationRelease_2019-06.xml"

    # first, load the latest clinvar from a static file
    with open(VARIATION_FILE, "rb") as fp:
        ctx = etree.iterparse(fp, events=("end",), tag="VariationArchive")

        for idx, (action, elem) in enumerate(ctx):
            if action == 'end':
                if str(elem.attrib['VariationID']) != str(variation_id):
                    continue

                return elem


In [ ]:
XML(get_variant(2))

In [11]:
render_tree(get_variant(16613))

# Extraction

In [ ]:
from lxml import etree
from collections import OrderedDict

In [ ]:
find_genes = etree.XPath("InterpretedRecord/*/GeneList/Gene")  # the symbol is the attribute 'Symbol'

In [ ]:
three_to_one = {
    '*': 'X',
    'Ter': 'X',  # this appears to be an NCBI-specific thing
    'Ala': 'A',
    'Arg': 'R',
    'Asn': 'N',
    'Asp': 'D',
    'Asx': 'B',
    'Cys': 'C',
    'Gln': 'Q',
    'Glu': 'E',
    'Glx': 'Z',
    'Gly': 'G',
    'His': 'H',
    'Ile': 'I',
    'Leu': 'L',
    'Lys': 'K',
    'Met': 'M',
    'Phe': 'F',
    'Pro': 'P',
    'Sec': 'U',
    'Ser': 'S',
    'Thr': 'T',
    'Trp': 'W',
    'Tyr': 'Y',
    'Val': 'V',
    'Xaa': 'Xa'
}

In [ ]:
def remap_prots(three_prot_change):
    """
    >>> remap_prots("Val600Glu")
    'V600E'
    """
    return reduce(lambda acc, x: acc.replace(x[0], x[1]), three_to_one.items(), three_prot_change)

### Aside: some xpath tests

In [ ]:
genes = q.xpath('InterpretedRecord/*/GeneList/Gene')

In [ ]:
[x.attrib for x in genes]

In [ ]:
q.xpath('InterpretedRecord/*/HGVSlist/HGVS[@Assembly="GRCh37" and @Type="genomic, top-level"]/NucleotideExpression/Expression/text()')

In [ ]:
q.xpath('InterpretedRecord/*/HGVSlist/HGVS/@Type')

In [ ]:
q.xpath('InterpretedRecord/*/HGVSlist/HGVS[@Type="coding"]/NucleotideExpression[starts-with(@sequenceAccessionVersion, "NM_")]/Expression/text()')




In [ ]:
q.find('RecordStatus').text

## Harvester's Expected Format

In [ ]:
# TODO: maybe later, extract structure out into something we can just apply to the root object
from functools import reduce

def unify(root):
    if root.find('RecordStatus').text != 'current' or root.find('Species').text != 'Homo sapiens':
        return
    
    genes = root.xpath('InterpretedRecord/*/GeneList/Gene')
    gene_symbols = [x.attrib['Symbol'] for x in genes]
    
    grch37_pos = root.xpath('//SimpleAllele/Location/SequenceLocation[@Assembly="GRCh37"]')[0]

    hgvs_g = root.xpath('InterpretedRecord/*/HGVSlist/HGVS[@Assembly="GRCh37" and @Type="genomic, top-level"]/NucleotideExpression/Expression/text()')[0]
    hgvs_c_first_node = root.xpath('InterpretedRecord/*/HGVSlist/HGVS[@Type="coding"]/NucleotideExpression')[0]
    hgvs_p_first_node = root.xpath('InterpretedRecord/*/HGVSlist/HGVS[@Type="protein"]/ProteinExpression')[0]
    
    prot_change = remap_prots(hgvs_p_first_node.attrib['change'][2:])
    
    return {
      'feature_association': {
        'source': 'clinvar',
        'source_url': 'https://www.ncbi.nlm.nih.gov/clinvar/variation/%d' % int(root.attrib['VariationID']), # should be the URL of the variant

        'genes': gene_symbols, # first element required
        # 'gene_identifiers': [
        # {
        # 'entrez_id': x.attrib['GeneID'],
        # 'ensembl_gene_id': 'required', # where can we get this? mygene.info?
        # 'uniprot_ids': 'required',
        # 'location': 'required',
        # 'symbol': 'required',
        # 'aliases': 'required',
        # 'prev_symbols': 'required'
        # }
        # for x in genes
        # ], # this is all filled out by the normalizer gene_enricher

        'features': [
          {
            'geneSymbol': gene_symbols[0],
            'name': prot_change,  # typically <PROTEIN-CHANGE>, e.g. "V600E"
            'sequence_ontology': { # optional, injected by biomarker_normalizer, relies on biomarker_type
              'soid': 'required',
              'so_name': 'required',
              'hierarchy': None
            },

            'description': "%s %s" % (gene_symbols[0], prot_change),  # typically <GENE-SYMBOL> <NAME>, e.g. "BRAF V600E"
            'referenceName': 'GRCh37',  # the assembly, in all cases so far GRCh37
            'refseq': hgvs_c_first_node.attrib['sequenceAccessionVersion'],
            'isoform': None,  # presumbly an ensembl accession, starts with ENST
            'biomarker_type': None,
            'chromosome': grch37_pos.attrib['Chr'],
            'start': grch37_pos.attrib['start'],
            'end': grch37_pos.attrib['stop'],
            'ref': grch37_pos.attrib['referenceAllele'],
            'alt': grch37_pos.attrib['alternateAllele'],
            'hgvs_g': hgvs_g,
            'hgvs_c': hgvs_c_first_node.xpath('Expression/text()')[0],
            'hgvs_p': hgvs_p_first_node.xpath('Expression/text()')[0],
            'dbsnp_ids': root.xpath('XRef[@Type="rs" and DB="dbSNP"]/@ID'),  
            'myvariant_hg19': None,  # injected by normalizer
            'mv_info': None,  # injected by normalizer
            'crawl_status': None  # used to report crawling issues
          }
        ],

        'association': {
          'description': None,
          'drug_labels': None,  # a comma-delimited list of drug names
          'drug_interaction_type': None,  # only supplied by CIViC afaict, and only ever 'Substitutes' or null
          'variant_name': None,
          'source_link': None,  # should be the URL of the evidence item(s)
          'evidence_type': None,  # one of (Prognostic, Predictive, Predisposing, Diagnostic, Functional)
          'evidence_direction': None,
          'clinical_significance': None,
          'evidence_level': None,
          'crawl_status': None,
          'extras': None,

          'phenotypes': [
            {
              'source': None,  # ontology link, e.g. http://purl.obolibrary.org/obo/DOID_4329
              'term': None,  # disease name, e.g. 'Erdheim-Chester disease'
              'id': None,  # some kind of ontology reference, e.g. 'DOID:4329'
              'family': None,  # e.g., 'histiocytosis'
              'description': None  # a readable version of the term, e.g. 'Erdheim-Chester disease'
            }
          ],

          'evidence': [
            {
              'info': { # optional
                'publications': None  # array, e.g. {http://www.ncbi.nlm.nih.gov/pubmed/26287849,http://www.ncbi.nlm.nih.gov/pubmed/29188284}
              },
              'evidenceType': { # optional
                'id': None,  # e.g., 'BRAF-Erdheim-Chester Disease' (gene name, disease name)
                'sourceName': 'clinvar'
              }
            }
          ],

          'environmentalContexts': [
            {
              'source': None,
              'term': None,  # presumably also a drug name, e.g., Vemurafenib, but also often null; no current entries differ from description
              'id': None,
              'usan_stem': None,
              'description': None,  # presumably a drug namem, e.g., Vemurafenib; no current entries differ from term
            }
          ]
        }
      }
    }


In [ ]:
unify(q)

## Specific Field Extraction

In [ ]:
# let's try to  get all the clinical assertions
interpretations = q.xpath('InterpretedRecord/Interpretations/Interpretation[@Type="Clinical significance"]')
interpretations

In [ ]:
interpretations[0].xpath('Description/text()')

In [ ]:
DISEASE_DB_URLS = {
    'OMIM': 'https://www.omim.org/entry/%s',
    'Genetic Alliance': 'https://www.diseaseinfosearch.org/%s',
    'OrphaNet': 'https://www.orpha.net/consor/cgi-bin/OC_Exp.php?Expert=%s',
    'SNOMED CT': 'http://purl.bioontology.org/ontology/SNOMEDCT/%s'
}

In [ ]:
def extract_phenotype(y):
    preferred_term = y.xpath('Name/ElementValue[@Type="Preferred"]')[0]
    xref = preferred_term.getparent().find('XRef')

    return {
        # ontology link, e.g. http://purl.obolibrary.org/obo/DOID_4329
        'source': DISEASE_DB_URLS[xref.attrib['DB']] % xref.attrib['ID'],
        # disease name, e.g. 'Erdheim-Chester disease'
        'term': preferred_term.text,
        # some kind of ontology reference, e.g. 'DOID:4329'
        'id': "%s:%s" % (xref.attrib['DB'], xref.attrib['ID']),
        # e.g., 'histiocytosis'
        'family': None,
        # a readable version of the term, e.g. 'Erdheim-Chester disease'
        'description': preferred_term.text
    }

In [ ]:
phenotypes = [
    extract_phenotype(y)
    for x in interpretations
    for y in x.xpath("ConditionList/TraitSet[@Type='Disease']/Trait[@Type='Disease']")
]
phenotypes

In [ ]:
evidence = [
    {
        'info': {  # optional
            'publications': []
            # array, e.g. {http://www.ncbi.nlm.nih.gov/pubmed/26287849,http://www.ncbi.nlm.nih.gov/pubmed/29188284}
        },
        'evidenceType': {  # optional
            'id': None,  # e.g., 'BRAF-Erdheim-Chester Disease' (gene name, disease name)
            'sourceName': 'clinvar'
        }
    }
    for x in interpretations
    for y in x.xpath("ConditionList/TraitSet[@Type='Disease']/Trait[@Type='Disease']")
]
evidence

# Aside: XML Output Formatting

In [ ]:
from display_xml import XML

In [ ]:
XML(q)

In [ ]:
from vdom import div
from format_tools import expando
def collapse_xml(tree):
    attribs = " ".join(('%s="%s"' % (k,v) for k,v in tree.attrib.items()))
    tree_rep = "%s %s" % (tree.tag, attribs)
    if len(tree):
        content = div(*(collapse_xml(elem) for elem in tree))
        return expando(u"▶ <%s />" % tree_rep, u"▼ <%s>" % tree_rep, content)
    else:
        return div("<%s />" % tree_rep)
    
    

In [ ]:
collapse_xml(q)